# Using the k-Nearest Neighbours Algorithm to Predict Bike Availability

The machine learning algorithm we decided to use in our project to predict bike availability was a tweaked version of the k-nearest neighbours algorithm. We have chosen this algorithm both for it's simplicity and, we would argue, it's suitability with some alteration for the task of predicting bike availability based on weather data. Our primary source of information for learning about this algorithm was the chapter 8.2 in Tom Mitchel's book "Machine Learning".

In the k-nearest neighbours algorithm, the instance we are trying to predict what a particular instance is based on the k (some number) "nearest neighbours" from out data set, that is to say the instances from our dataset that our closest to the instance being predicted in it's known qualities. This algorithm is usually used to allocate instances into discrete categories, in which cases the instance will be given the same classification as the majority of it's k-nearest neighbours. As the predictions we are assigning are continuous, we will use the average of the k-nearest neighbours, i.e. the average available bikes of the nearest neighbours selected based on similarity of weather conditions. To maximise accuracy, we will be implementing a version of k nearest neighbours outlined by Mitchell on pp.233-4, distance weighted nearest neighbours where k is all training examples. In terms of our project, this means our prediction will be based on the weighted average of all of the previous bike availability data for a station, with examples having a higher weight in the average the closer they are in weather to weather for our prediction.  

In [1]:
import mysql.connector
import math
from datetime import datetime, timedelta
import requests
from random import randint
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'mysql'

### Prediction Model

In [ ]:
def nearest_neighbours(station_no):
    data = requests.get("http://api.openweathermap.org/data/2.5/forecast?q=Dublin&appid=6fb76ecce41a85161d4c6ea5e2758f2b").json()
    mydb = mysql.connector.connect(
        host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
        user="root",
        passwd="secretpass"
    )

    cursor = mydb.cursor(buffered=True)

    counter = 0
    
    predictions = []

    for forecast in data['list']: #retrieving the data and time information from the api call to display
        dt = forecast['dt']
        dt = int(dt)
        dt = datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
        date, time = dt.split(" ")

        #for time in dates_and_times[date]:

        forecast_key =  date + " " + time

        prediction_date = datetime.strptime(date, '%Y-%m-%d')

        prediction_day = prediction_date.weekday()

        dt_time = datetime.strptime(time, '%H:%M:%S')

        prediction_time = timedelta(hours=dt_time.hour, minutes=dt_time.minute, seconds=dt_time.second)

        prediction_temp = forecast['main']['temp']

        prediction_weather = forecast['weather'][0]['main']
        
        weight_total = 0 
        weighted_predictors_total = 0 
        
        #now that we have a prediction for weather for that particular time and date, we can compare it to our previous records
        
        #if the prediction is on a weekday, we only use weekday records
        if (prediction_day < 5):
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) < 5" % (station_no, prediction_weather))
            #for the station number in question we are retrieving all of our records where the general weather description is the same (raining, clouds, etc.) and the time of day is roughly the same, that is to say less than 11 minutes off. We will not be lookng at records where the station was not open.
            rows = cursor.fetchall()
            if len(rows) == 0: #if a currently unknown weather is encountered (one there is not previous data on), we will do the same as above but for all weather description types, i.e. if snow is encountered for the first time we will take records with rainy, clear, clouds, mist, drizzle and any other weather types
                cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' weekday(weather.date) < 5" % station_no)
                rows = cursor.fetchall()
        #if weekend, use only records from that day:
        else:
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) = %s" % (station_no, prediction_weather, prediction_day))
            rows = cursor.fetchall()
            if len(rows) == 0: #if a currently unknown weather is encountered (one there is not previous data on), we will do the same as above but for all weather description types, i.e. if snow is encountered for the first time we will take records with rainy, clear, clouds, mist, drizzle and any other weather types
                cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 15 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weekday(weather.date) = %s" % station_no, prediction_day)
                rows = cursor.fetchall()
        #we will now get the weighted average of all the records retrieved above   
        for row in rows:
            row_temp = row[6]
            row_bikes = row[2]
            row_time = row[8]
            temp_weight = 1/(math.sqrt((row_temp - prediction_temp)**2) + 0.5) #the difference between the temperature in a record and the predicted temperature
            time_weight = 1/(math.sqrt((round((row_time - prediction_time).total_seconds()/60)**2)) + 0.5) #the difference between the time in a record and the time of the prediction
            weight = temp_weight + time_weight #weight is determined based on the difference in both time and temperature
            #adding the weight and weighted predictions from this record to the totals
            weight_total += weight   
            weighted_predictor = row_bikes * weight
            weighted_predictors_total += weighted_predictor
        #finally, our prediction is the waited average available bikes from the records we retrieved
        prediction = round(weighted_predictors_total/weight_total)
        #print(prediction)
        predictions.append(prediction)
        counter += 1
        if counter == 9:
            break

    return predictions
        

In [ ]:
%%time
prediction = nearest_neighbours(2)

In [ ]:
print("Available bikes prediction for station 2 is:", prediction)

## Accuracy Evaluation

The method below evaluates the average accuracy of the predictive model for a given number of random tests using past data:

In [ ]:
mydb = mysql.connector.connect(
        host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
        user="root",
        passwd="secretpass"
    )

cursor = mydb.cursor(buffered=True)

In [ ]:
cursor.execute("SELECT * FROM innodb.weather")

#cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather ON station_var.last_update_date = weather.date WHERE (minute(timediff(station_var.lat_update_time, weather.time)) < 1 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) AND station_var.status = 'OPEN' LIMIT 100;")

#joined_rows = cursor.fetchall()

#df = pd.DataFrame(joined_rows)

weather_rows = cursor.fetchall()

weather_df = pd.DataFrame(weather_rows)

print(weather_df.head())

print("----------------------------")

print(weather_df.shape)

In [ ]:
weather_df.columns = ['temp', 'weather', 'time', 'date']
print(weather_df.head())

In [ ]:
cursor.execute("SELECT * FROM innodb.station_var")
station_rows = cursor.fetchall()
station_df = pd.DataFrame(station_rows)
print(station_df.head())
print("--------------------")
print(station_df.shape)

In [ ]:
station_df.columns = ['status', 'stands', 'bikes', 'date', 'station_time', 'station_no']
print(station_df.head())

In [ ]:
station_df = station_df[(station_df.status == 'OPEN')]
station_df = station_df.drop("status", axis=1)
station_df = station_df.drop("stands", axis=1)
merged_df = weather_df.merge(station_df, on='date')
print(merged_df.head())
print(merged_df.shape[0])

In [ ]:
df = merged_df[(abs(merged_df.time - merged_df.station_time) < pd.to_timedelta('00:15:00'))]
df = df.drop_duplicates()
print(df[(df.station_no == 42)].tail())
print(df.shape[0])

In [ ]:
def check_accuracy(no_tests, df):

    train_df, test_df = train_test_split(df, test_size=0.3)
    
    train_df['date'] = pd.to_datetime(train_df['date'])
        
    tests_completed = 0
    
    #total_prediction_error = 0
    
    #joined_rows_count = len(joined_rows)
    
    no_test_rows = test_df.shape[0]
    
    prediction_list = []
    
    true_list = []
    
    while tests_completed < no_tests: #repeat below for the given amount of tests to complete
        
        #station_no = station_no_rows[random.randint(0, station_no_rows_count - 1)][0] #select a random station number
        
        #while test_row[3] == date:
        #test_row = test_df.sample() #select a random weather record
        
        #print(test_row)
        
        #return
        
        random_row = randint(0, no_test_rows - 1)

        temp = test_df.iloc[random_row]['temp'] #storing the data from that weather record

        weather = test_df.iloc[random_row]['weather'] 
        
        time = test_df.iloc[random_row]['time']
        
        date = test_df.iloc[random_row]['date']
        
        day = date.weekday()
        
        bikes_available = test_df.iloc[random_row]['bikes']
        
        true_list.append(bikes_available)
        
        station_no = test_df.iloc[random_row]['station_no']
        
        '''
        print("Time:", time)
        print("Temp:", temp)
        print("Weather:", weather)
        print("Date:", date)
        print("Day:", day)
        print("Station:", station_no)
        print("Bikes:", bikes_available)
        
        '''
        weight_total = 1 
        weighted_predictors_total = 1 
        if (day < 5):   
            train_rows = train_df[(train_df['weather'] == weather) & (train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday < 5)]
            if train_rows.empty:  
                train_rows = train_df[(train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday < 5)]
        else:
            train_rows = train_df[(train_df['weather'] == weather) & (train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday == day)]
            if train_rows.empty:  
                train_rows = train_df[(train_df['station_no'] == station_no) & (abs(train_df['time'] - time) < pd.to_timedelta('00:15:00')) & (train_df['date'].dt.weekday == day)]
            #print(train_rows.head())
        for i in range(train_rows.shape[0] - 1):
            #print("hi")
            row_temp = train_rows.iloc[i]['temp']
            row_bikes = train_rows.iloc[i]['bikes']
            row_time = train_rows.iloc[i]['time']
            #print(row_temp)
            #print(row_bikes)
            #print(row_time)
            #print()
            #print(round((row_time - time).total_seconds()))
            #return
            temp_weight = 1/(abs(row_temp - temp) + 0.5)
            time_weight = 1/((abs(row_time - time).total_seconds()/60) + 0.5)
            weight = temp_weight + time_weight 
            weight_total += weight
            #print(weight_total)
            weighted_predictor = row_bikes * weight
            weighted_predictors_total += weighted_predictor
        print("----------------------TEST " + str(tests_completed + 1) + "----------------------")
        print("Station no:", station_no)
        if weight_total != 0:
            prediction = round(weighted_predictors_total/weight_total)
            prediction_list.append(prediction)
            print("Predicted bikes available:", prediction_list[tests_completed])
            print("Bikes actually available:", true_list[tests_completed])
            tests_completed += 1
        else:
            print("No suitable rows to make a prediction.")
            true_list.pop()
        print("--------------------------------------------------")
        print()
        
        
    print("------------------TESTS FINISHED------------------")
    return [true_list, prediction_list]

In [ ]:
metrics = check_accuracy(5000, df)

In [ ]:
true = metrics[0]
#print(true)
prediction = metrics[1]
#print(prediction)

In [ ]:
#print("Accuracy score:", accuracy_score(true, prediction, normalize = False))
print("Normalized accuracy score:", accuracy_score(true, prediction))

### Measuring and Improving Performance

Our function already seems quite fast, but we will now see exactly how fast it is and see if it's performance can be optimised.

In [ ]:
import time as stopwatch

In [ ]:
def timer(station_no):
    data = requests.get("http://api.openweathermap.org/data/2.5/forecast?q=Dublin&appid=6fb76ecce41a85161d4c6ea5e2758f2b").json()

    mydb = mysql.connector.connect(
        host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
        user="root",
        passwd="secretpass"
    )

    cursor = mydb.cursor(buffered=True)

    forecasts = {}
    dates = []
    dates_and_times = {}

    for forecast in data['list']:
        dt = forecast['dt']
        dt = int(dt)
        dt = datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
        date, time = dt.split(" ")
        forecasts[dt] = forecast
        if date not in dates:
            dates.append(date)
            dates_and_times[date] = []
        dates_and_times[date].append(time)

    print()

    for key in dates_and_times:
        print(key)

    print() 

    date = input("Please enter which of the above dates you would like to predict bike availability on: ")

    print() 

    for time in dates_and_times[date]:
        print(time)

    print() 

    time = input("Please enter one of the above times to predict bike availability on the selected date for: ")

    start = stopwatch.time() #starting the 'stopwatch' after last user input has been received
    
    forecast_key =  date + " " + time

    prediction_date = datetime.strptime(date, '%Y-%m-%d')

    prediction_day = prediction_date.weekday()

    dt_time = datetime.strptime(time, '%H:%M:%S')

    prediction_time = timedelta(hours=dt_time.hour, minutes=dt_time.minute, seconds=dt_time.second)

    prediction_temp = forecasts[forecast_key]['main']['temp']

    prediction_weather = forecasts[forecast_key]['weather'][0]['main']

    if (prediction_day < 5):
        cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) < 5" % (station_no, prediction_weather))
        rows = cursor.fetchall()
        if rows == []: #if a new weather is encountered, use records for all weather
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' weekday(weather.date) < 5" % station_no)
            rows = cursor.fetchall()
        weight_total = 0 
        weighted_predictors_total = 0  
        for row in rows:
            row_temp = row[6]
            row_bikes = row[2]
            row_time = row[8]
            temp_weight = 1/(math.sqrt((row_temp - prediction_temp)**2) + 0.5)
            time_weight = 1/(math.sqrt((round((row_time - prediction_time).total_seconds()/60)**2)) + 0.5)
            weight = temp_weight + time_weight 
            weight_total += weight
            weighted_predictor = row_bikes * weight
            weighted_predictors_total += weighted_predictor
       

    #if weekend, use only records from that day:
    else:
        cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) = %s" % (station_no, prediction_weather, prediction_day))
        rows = cursor.fetchall()
        if rows == []: #if a new weather is encountered, use records for all weather
            #print("New weather encountered:", prediction_weather)
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weekday(weather.date) = %s" % (station_no, prediction_day))
            rows = cursor.fetchall()
        weight_total = 0 
        weighted_predictors_total = 0  
        for row in rows:
            row_temp = row[6]
            row_bikes = row[2]
            row_time = row[8]
            temp_weight = 1/(math.sqrt((row_temp - prediction_temp)**2) + 0.5)
            time_weight = 1/(math.sqrt((round((row_time - prediction_time).total_seconds()/60)**2)) + 0.5)
            weight = temp_weight + time_weight 
            weight_total += weight
            weighted_predictor = row_bikes * weight
            weighted_predictors_total += weighted_predictor
            
    prediction = round(weighted_predictors_total/weight_total)
    
    stop = stopwatch.time() 
    
    return stop - start #returning how long after the last user input the function took to run
        

In [ ]:
timer(2)

In [ ]:
timer(105)

In [ ]:
timer(51)

In [ ]:
timer(68)

In [ ]:
timer(97)

A prediction usually seems to be received in around a second or less, going slightly over in one case. This seems like acceptable performance, but perhaps we could improve it through the use of generators:

In [ ]:
#source for code in this cell: http://code.activestate.com/recipes/137270-use-generators-for-fetching-large-db-record-sets/

from __future__ import generators

def ResultIter(cursor, arraysize=1000):
    'An iterator that uses fetchmany to keep memory usage down'
    while True:
        results = cursor.fetchmany(arraysize)
        if not results:
            break
        for result in results:
            yield result

In [ ]:
def function_with_generators(station_no):
    data = requests.get("http://api.openweathermap.org/data/2.5/forecast?q=Dublin&appid=6fb76ecce41a85161d4c6ea5e2758f2b").json()

    mydb = mysql.connector.connect(
        host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
        user="root",
        passwd="secretpass"
    )

    cursor = mydb.cursor(buffered=True)

    forecasts = {}
    dates = []
    dates_and_times = {}

    for forecast in data['list']:
        dt = forecast['dt']
        dt = int(dt)
        dt = datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
        date, time = dt.split(" ")
        forecasts[dt] = forecast
        if date not in dates:
            dates.append(date)
            dates_and_times[date] = []
        dates_and_times[date].append(time)

    print()

    for key in dates_and_times:
        print(key)

    print() 

    date = input("Please enter which of the above dates you would like to predict bike availability on: ")

    print() 

    for time in dates_and_times[date]:
        print(time)

    print() 

    time = input("Please enter one of the above times to predict bike availability on the selected date for: ")

    start = stopwatch.time() #starting the 'stopwatch' after last user input has been received
    
    forecast_key =  date + " " + time

    prediction_date = datetime.strptime(date, '%Y-%m-%d')

    prediction_day = prediction_date.weekday()

    dt_time = datetime.strptime(time, '%H:%M:%S')

    prediction_time = timedelta(hours=dt_time.hour, minutes=dt_time.minute, seconds=dt_time.second)

    prediction_temp = forecasts[forecast_key]['main']['temp']

    prediction_weather = forecasts[forecast_key]['weather'][0]['main']
    
    class global_vars():
        weighted_predictors_total = 0
        weight_total = 0
    
    def add_predictor(row):
        #row_temp = row[6] #skip the assignments for the sake of speed
        #row_bikes = row[2] 
        #row_time = row[8]
        temp_weight = 1/(math.sqrt((row[6] - prediction_temp)**2) + 0.5)
        time_weight = 1/(math.sqrt((round((row[8] - prediction_time).total_seconds()/60)**2)) + 0.5)
        weight = temp_weight + time_weight 
        global_vars.weight_total += weight
        weighted_predictor = row[2] * weight
        global_vars.weighted_predictors_total += weighted_predictor
               

    if (prediction_day < 5):
        cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) < 5" % (station_no, prediction_weather))
        #rows = cursor.fetchall()
        #if rows == []: #if a new weather is encountered, use records for all weather
            #cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' weekday(weather.date) < 5" % station_no)
            #rows = cursor.fetchall()
        for result in ResultIter(cursor):
            add_predictor(result)
       
        
       

    #if weekend, use only records from that day:
    else:
        cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) = %s" % (station_no, prediction_weather, prediction_day))
        #rows = cursor.fetchall()
        #if rows == []: #if a new weather is encountered, use records for all weather
            #cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weekday(weather.date) = %s" % (station_no, prediction_day))
            #rows = cursor.fetchall()
        for result in ResultIter(cursor):
            add_predictor(result)

    print(global_vars.weighted_predictors_total)
    
    print(global_vars.weight_total)
    
    prediction = round(global_vars.weighted_predictors_total/global_vars.weight_total)
    
    print("Available bikes prediction for station 2 is:", prediction)
    
    stop = stopwatch.time() 
    
    return stop - start #returning how long after the last user input the function took to run
        

In [ ]:
function_with_generators(2)

In [ ]:
function_with_generators(105)

It seems that adding generators has actually made the function slightly slower, unfortunately. 

We will now test the our predictive function without generators but with the other optimisations included in function_with_generators (less assignments).

In [ ]:
def optimised_function(station_no):
    data = requests.get("http://api.openweathermap.org/data/2.5/forecast?q=Dublin&appid=6fb76ecce41a85161d4c6ea5e2758f2b").json()

    mydb = mysql.connector.connect(
        host="newdublinbikesinstance.cevl8km57x9m.us-east-1.rds.amazonaws.com",
        user="root",
        passwd="secretpass"
    )

    cursor = mydb.cursor(buffered=True)

    forecasts = {}
    dates = []
    dates_and_times = {}

    for forecast in data['list']:
        dt = forecast['dt']
        dt = int(dt)
        dt = datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
        date, time = dt.split(" ")
        forecasts[dt] = forecast
        if date not in dates:
            dates.append(date)
            dates_and_times[date] = []
        dates_and_times[date].append(time)

    print()

    for key in dates_and_times:
        print(key)

    print() 

    date = input("Please enter which of the above dates you would like to predict bike availability on: ")

    print() 

    for time in dates_and_times[date]:
        print(time)

    print() 

    time = input("Please enter one of the above times to predict bike availability on the selected date for: ")

    start = stopwatch.time() #starting the 'stopwatch' after last user input has been received
    
    forecast_key =  date + " " + time

    prediction_date = datetime.strptime(date, '%Y-%m-%d')

    prediction_day = prediction_date.weekday()

    dt_time = datetime.strptime(time, '%H:%M:%S')

    prediction_time = timedelta(hours=dt_time.hour, minutes=dt_time.minute, seconds=dt_time.second)

    prediction_temp = forecasts[forecast_key]['main']['temp']

    prediction_weather = forecasts[forecast_key]['weather'][0]['main']

    if (prediction_day < 5):
        cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) < 5" % (station_no, prediction_weather))
        rows = cursor.fetchall()
        if rows == []: #if a new weather is encountered, use records for all weather
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' weekday(weather.date) < 5" % station_no)
            rows = cursor.fetchall()
        weight_total = 0 
        weighted_predictors_total = 0  
        for row in rows:
            #row_temp = row[6] #skip the assignments for the sake of speed
            #row_bikes = row[2] 
            #row_time = row[8]
            temp_weight = 1/(math.sqrt((row[6] - prediction_temp)**2) + 0.5)
            time_weight = 1/(math.sqrt((round((row[8] - prediction_time).total_seconds()/60)**2)) + 0.5)
            weight = temp_weight + time_weight 
            weight_total += weight
            weighted_predictor = row[2] * weight
            weighted_predictors_total += weighted_predictor
       

    #if weekend, use only records from that day:
    else:
        cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weather.description = '%s' AND weekday(weather.date) = %s" % (station_no, prediction_weather, prediction_day))
        rows = cursor.fetchall()
        if rows == []: #if a new weather is encountered, use records for all weather
            #print("New weather encountered:", prediction_weather)
            cursor.execute("SELECT DISTINCT * FROM innodb.station_var JOIN innodb.weather on (station_var.last_update_date = weather.date AND minute(timediff(station_var.lat_update_time, weather.time)) < 11 AND hour(timediff(station_var.lat_update_time, weather.time)) = 0) WHERE station_var.station_no = %s AND station_var.status = 'OPEN' AND weekday(weather.date) = %s" % (station_no, prediction_day))
            rows = cursor.fetchall()
        weight_total = 0 
        weighted_predictors_total = 0  
        for row in rows:
            #row_temp = row[6] #skip the assignments for the sake of speed
            #row_bikes = row[2] 
            #row_time = row[8]
            temp_weight = 1/(math.sqrt((row[6] - prediction_temp)**2) + 0.5)
            time_weight = 1/(math.sqrt((round((row[8] - prediction_time).total_seconds()/60)**2)) + 0.5)
            weight = temp_weight + time_weight 
            weight_total += weight
            weighted_predictor = row[2] * weight
            weighted_predictors_total += weighted_predictor
            
    prediction = round(weighted_predictors_total/weight_total)
    
    stop = stopwatch.time() 
    
    return stop - start #returning how long after the last user input the function took to run
        

In [ ]:
optimised_function(2)

In [ ]:
optimised_function(105)

Reducing the number of assignments doesn't seem to have made much of a difference to the runtime, but we will use this for the final version as it should technically be slightly faster due to containing less operations. The function was sufficiently fast from the beginning anyway, so while further optimisation would have been ideal, it is not really needed. 